# BDA1 - Spark

In this set of exercises you will work exclusively with Spark. This means that in your programs, you only need to create the `SparkContext`.
In a number of exercises you will be asked to calculated temperature averages (daily and monthly). These are not always computed according to the standard definition of ‘average’. In this domain the daily average temperature is calculated by averaging the daily measured maximum and the daily measured minimum temperatures. The monthly average is calculated by averaging the daily maximums and minimums for that month. For example, to get the monthly average for October, take maximums and minimums for each day, sum them up and divide by 62 (which is the same as taking the daily averages, summing them up and divide by the number of days).

Note: In many countries in the world, the averages are calculated as discussed. However, in Sweden, daily and monthly averages are calculated using Ekholm-Modéns formula which in addition to minimum and maximum daily temperature also takes into account readings at specific timepoints, the month as well as the longitude of the station. For more information check (in Swedish): http://www.smhi.se/kunskapsbanken/meteorologi/hur-beraknas-medeltemperatur-1.3923

In [7]:
import pyspark
import random
import csv
import math
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

tr_path = "/home/flennic/Documents/big-data-analytics-ressources/temperature-readings.csv"
pr_path = "/home/flennic/Documents/big-data-analytics-ressources/precipitation-readings.csv"
sro_path = "/home/flennic/Documents/big-data-analytics-ressources/stations-Ostergotland.csv"
sr_path = "/home/flennic/Documents/big-data-analytics-ressources/stations.csv"

In [2]:
sc = pyspark.SparkContext(appName = "Temperature")
temperature_readings = sc.textFile(tr_path)
precipitation_readings = sc.textFile(pr_path)
station_oster_readings = sc.textFile(sro_path)
station_readings = sc.textFile(sr_path)

# Assignment 1

**Task 1a:** What are the lowest and highest temperature measured each year for the period 1950 to 2014? Provide the list sorted in the descending order with respect to the maximum temperature. Extend the program to include the station number (not the station name) where the maximum/minimum temperature was measured.

**Answer:** As the exercise 1a) is a special case of 1), we combine them to save computation time.

In [ ]:
#%%time
global res
res = temperature_readings.map(lambda l: l.split(";"))
res = res.map(lambda l: (int(l[1][0:4]), (int(l[0]), float(l[3]))))
res = res.filter(lambda m: m[0] >= 1950 and m[0] <= 2014)

res_max = res.reduceByKey(lambda x, y: max(x, y, key=lambda z: float(z[1])))
res_max = res_max.map(lambda m: (m[0], m[1][1], m[1][0]))
res_min = res.reduceByKey(lambda x, y: min(x, y, key=lambda z: float(z[1])))
res_min = res_min.map(lambda m: (m[1][1], m[1][0]))

res = res_max.zip(res_min).map(lambda x: [m for sublist in x for m in sublist])
res = res.sortBy(lambda m: float(m[1]), ascending=False)

global df
df = pd.DataFrame(res.collect(), columns=['year', 'temp_max', 'station_max', 'temp_min', 'station_min'])

In [ ]:
df.head()

**Task 1b**: Same exercise, this time as a non-parallel version.

In [ ]:
def csv_read_lazy(csvfile, delimiter = ",", encoding = "utf-8"):
    with open(csvfile, encoding = encoding) as f:
        r = csv.reader(f, delimiter = delimiter)
        for row in r:
            yield row

In [ ]:
%%time

def get_temperature_information(min_year = 1950, max_year = 2014):
    
    temp_info = {}
    temp_info["max"] = {}
    temp_info["min"] = {}
    
    for year in range(min_year, max_year + 1):
        temp_info["max"][year] = {"temp": float("-Inf"), "station": None}
        temp_info["min"][year] = {"temp": float("Inf"), "station": None}

    for row in csv_read_lazy(tr_path, delimiter = ";"):
        year = int(row[1][0:4])
        station = int(row[0])
        temp = float(row[3])
    
        if year >= min_year and year <= max_year:
            # Max
            if temp > temp_info["max"][year]["temp"]:
                temp_info["max"][year]["temp"] = temp
                temp_info["max"][year]["station"] = station
            
            # Min
            if temp < temp_info["min"][year]["temp"]:
                temp_info["min"][year]["temp"] = temp
                temp_info["min"][year]["station"] = station
                
    df = pd.DataFrame(columns=['year', 'temp_max', 'station_max', 'temp_min', 'station_min'])
        
    for year, v in temp_info["max"].items():
        if v["temp"] != float("-Inf"):
            df.loc[len(df)] = [year, v["temp"], v["station"], temp_info["min"][year]["temp"], temp_info["min"][year]["station"]]
    
    df["year"] = df["year"].astype(int)
    df["station_max"] = df["station_max"].astype(int)
    df["station_min"] = df["station_min"].astype(int)
    
    return df.sort_values(by=['temp_max'], ascending=False)

global df
df = get_temperature_information()

In [ ]:
df.head(10)

# Assignment 2

**Task 2:** Count the number of readings for each month in the period of 1950-2014 which are higher than 10 degrees. Repeat the exercise, this time taking only distinct readings from each station. That is, if a station reported a reading above 10 degrees in some month, then it appears only once in the count for that month. In this exercise you will use the `temperature-readings.csv` file.

The output should contain the following information:

Year, month, count

In [ ]:
%%time
res = temperature_readings.map(lambda l: l.split(";"))

res = res.map(lambda l: (int(l[1][0:4]), int(l[1][5:7]), float(l[3])))
res = res.filter(lambda m: int(m[0]) >= 1950 and int(m[0]) <= 2014 and m[2] > 10.0)
res = res.map(lambda m: (str(m[0]) + "-" + str(m[1]).zfill(2), 1))
res = res.reduceByKey(lambda x, y: x + y)
res = res.sortBy(lambda m: m[0], ascending=False)

global df
df = pd.DataFrame(res.collect(), columns=['year-month', 'count'])

In [ ]:
df.head(10)

Now we repeat the exercise where we take only distinct readings from each station.

In [ ]:
%%time

res = temperature_readings.map(lambda l: l.split(";"))
res = res.map(lambda l: (int(l[0]), int(l[1][0:4]), int(l[1][5:7]), float(l[3])))
res = res.filter(lambda m: int(m[1]) >= 1950 and int(m[1]) <= 2014 and m[3] > 10.0)
res = res.map(lambda m: (str(m[0]) + "-" + str(m[1]) + "-" + str(m[2]).zfill(2), 1))
res = res.reduceByKey(lambda x, y: x + y)
res = res.map(lambda m: (m[0][-7:], 1))
res = res.reduceByKey(lambda x, y: x + y)
res = res.sortBy(lambda m: m[0], ascending=False)
res = res.map(lambda m: (m[0][0:4], m[0][5:7], m[1]))

global df
df = pd.DataFrame(res.collect(), columns=['year', 'month', 'count'])

In [ ]:
df.head(10)

# Assignment 3

Find the average monthly temperature for each available station in Sweden. Your result should include average temperature for each station for each month in the period of 1960-2014. Bear in mind that not every station has the readings for each month in this timeframe. In this exercise you will use the `temperature-readings.csvfile`.

The output should contain the following information:

Year, month, station number, average monthly temperature

In [ ]:
%%time

res = temperature_readings.map(lambda l: l.split(";"))
res = res.map(lambda m: (m[0] + "-" + m[1][0:7], int(m[1][0:4]), float(m[3])))
res = res.filter(lambda m: m[1] >= 1960 and m[1] <= 2014)
res = res.map(lambda m: (m[0], (m[2], 1)))
res = res.reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))
res = res.map(lambda m: (int(m[0][:-8]), int(m[0][-7:-3]), str(m[0][-2:]).zfill(2), round(m[1][0]/m[1][1], 2)))
res = res.sortBy(lambda m: (m[1], m[2], m[0]), ascending=False)

global df
df = pd.DataFrame(res.collect(), columns=['station', 'year', 'month', 'average-temperature'])

In [ ]:
df.head(10)

# Assignment 4

Provide  a  list  of  stations  with  their  associated  maximum  measured  temperatures  and maximum  measured  daily  precipitation.  Show  only  those  stations  where  the  maximum temperature is between 25 and 30 degrees and maximum daily precipitation is between 100 mm and 200mm. In this exercise you will use the `temperature-readings.csv` and `precipitation-readings.csv` files. 

The output should contain the following information:

Station number, maximum measured temperature, maximum daily precipitation

In [ ]:
%%time

res_p = precipitation_readings.map(lambda l: l.split(";"))
res_t = temperature_readings.map(lambda l: l.split(";"))

res_p = res_p.map(lambda m: (int(m[0]), float(m[3])))
res_p = res_p.filter(lambda m: m[1] > 100 and m[1] < 200)
res_p = res_p.reduceByKey(lambda x, y: max(x, y))

res_t = res_t.map(lambda m: (int(m[0]), float(m[3])))
res_t = res_t.filter(lambda m: m[1] > 20 and m[1] < 30)
res_t = res_t.reduceByKey(lambda x, y: max(x, y))

res = res_t.join(res_p)
res = res.map(lambda m: (m[0], m[1][0], m[1][1]))

global df
df = pd.DataFrame(res.collect(), columns=['station', 'max_temperature', 'max_precipitation'])

In [ ]:
df.head()

# Assignment 5

Calculate the average monthly precipitation for the Östergotland region (list of stations is provided in the separate file) for the period 1993-2016. In order to do this, you will first need to calculate  the  total  monthly  precipitation  for  each  station  before calculating the monthly average (by averaging over stations). In this exercise you will use the `precipitation-readings.csv` and `stations-Ostergotland.csv` files.

HINT (not for the SparkSQL lab): Avoid using joins here! `stations-Ostergotland.csv` is small  and if distributed will cause a number of unnecessary shuffles when joined with precipitation RDD. If you distribute `precipitation-readings.csv` then either repartition your stations RDD to 1 partition or make use of the collect to acquire a python list and broadcast function to broadcast the list to all nodes.

The output should contain the following information:

Year, month, average monthly precipitation

In [3]:
%%time

res_p = precipitation_readings.map(lambda l: l.split(";"))
res_s = station_oster_readings.map(lambda l: l.split(";"))

res_p = res_p.map(lambda m: (int(m[0]), m[1], float(m[3])))
res_p = res_p.filter(lambda m: int(m[1][0:4]) >= 1993 and int(m[1][0:4]) <= 2016)

res_s = res_s.map(lambda m: int(m[0])).collect()
res_s = sc.broadcast(value=res_s)

res = res_p.filter(lambda m: m[0] in res_s.value)
res = res.map(lambda m: (str(m[0]) + "-" + m[1][0:7], (m[2], 1)))
res = res.reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))

res = res.map(lambda m: (m[0][6:10] + "-" + m[0][11:13], (m[1][0]/m[1][1], 1)))
res = res.reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))
res = res.map(lambda m: (int(m[0][0:4]), m[0][5:8], m[1][0]/m[1][1]))
res = res.sortBy(lambda m: (m[0], m[1]), ascending=False)

global df
df = pd.DataFrame(res.collect(), columns=['year', 'month', 'average_precipitation'])

In [13]:
df.tail(10)

,year,month,average_precipitation
270,1994,01,0.040625
271,1993,12,0.051671
272,1993,11,0.060537
273,1993,10,0.058143
274,1993,09,0.060237
275,1993,08,0.114144
276,1993,07,0.128745
277,1993,06,0.093234
278,1993,05,0.028514
279,1993,04,0.000000


In [ ]:
# Data for plotting
#for index in range(0, df.shape[0]):
#    print(df.iloc[index,])

#x = df['month'].head()
#y = 

#fig, ax = plt.subplots()
#ax.plot(y, x)

#ax.set(xlabel='time (s)', ylabel='voltage (mV)',
#       title='About as simple as it gets, folks')
#ax.grid()

#fig.savefig("test.png")
#plt.show()

# Assignment 6

Compare the average monthly temperature (find the difference) in the period 1950-2014 for all stations in Östergotland with long-term monthly averages in the period of 1950-1980. Make a plot of your results. 

HINT: The first step is to find the monthly averages for each station. Then, you can average over all stations to acquire the average temperature for a specific year and month. This RDD/DataFrame can beu sed to compute the long-term average by averaging over all the years in the interval.

The out put should contain the following information:

Year, month, difference

In [ ]:
sc.stop()